## CUDA events for both GPU timing and overlapping CPU and GPU execution. 

Events are inserted into a stream
 * of CUDA calls.  Since CUDA stream calls are asynchronous, the CPU can
 * perform computations while GPU is executing (including DMA memcopies
 * between the host and device).  CPU can query CUDA events to determine
 * whether GPU has completed tasks.

 Since CUDA stream calls are asynchronous, the CPU can
 * perform computations while GPU is executing (including DMA memcopies
 * between the host and device). 

// spin lock?
  CPU can query CUDA events to determine
 * whether GPU has completed tasks.

In [ ]:
// This will output the proper CUDA error strings 
in the event
// that a CUDA host call returns an error
#define checkCudaErrors(val)

cudaError_t 

In [ ]:
//malloc 对

  int n = 16 * 1024 * 1024;//16mb
  int nbytes = n * sizeof(int);
  int value = 26;

  // allocate host memory
  int *a = 0;
  checkCudaErrors(cudaMallocHost((void **)&a, nbytes));
  memset(a, 0, nbytes);

  // allocate device memory
  int *d_a = 0;
  checkCudaErrors(cudaMalloc((void **)&d_a, nbytes));
  checkCudaErrors(cudaMemset(d_a, 255, nbytes));


//=============release resorces===================
  checkCudaErrors(cudaFreeHost(a));
  checkCudaErrors(cudaFree(d_a));


In [ ]:
  // set kernel launch configuration
  dim3 threads = dim3(512, 1);
  dim3 blocks = dim3(n / threads.x, 1);


dim3

stream 0 
    events

In [ ]:
//events 对

//create cuda event  handles
  // create cuda event handles
  cudaEvent_t start, stop;
  //checkCudaErrors(cudaEventCreate(&start));
  //checkCudaErrors(cudaEventCreate(&stop));

  //StopWatchInterface *timer = NULL;
  //sdkCreateTimer(&timer);
  //sdkResetTimer(&timer);

  checkCudaErrors(cudaDeviceSynchronize());//block until completed all tasks
  float gpu_time = 0.0f;


  // asynchronously issue work to the GPU (all to stream 0)
  checkCudaErrors(cudaProfilerStart());
  //sdkStartTimer(&timer);


  
  cudaEventRecord(start, 0);
      cudaMemcpyAsync(d_a, a, nbytes, cudaMemcpyHostToDevice, 0);
      increment_kernel<<<blocks, threads, 0, 0>>>(d_a, value);
      cudaMemcpyAsync(a, d_a, nbytes, cudaMemcpyDeviceToHost, 0);
  cudaEventRecord(stop, 0);

  //sdkStopTimer(&timer);
  checkCudaErrors(cudaProfilerStop());

In [ ]:
//自旋锁等待，处理其他事项
  // have CPU do some work while waiting for stage 1 to finish
  unsigned long int counter = 0;

  while (cudaEventQuery(stop) == cudaErrorNotReady) {
    counter++;
  }

  checkCudaErrors(cudaEventElapsedTime(&gpu_time, start, stop));


In [ ]:

  // print the cpu and gpu times
  printf("time spent executing by the GPU: %.2f ms\n", gpu_time);
  printf("time spent by CPU in CUDA calls: %.2f ms\n", sdkGetTimerValue(&timer));
  printf("CPU executed %lu iterations while waiting for GPU to finish\n",
         counter);


In [ ]:

  // check the output for correctness
  bool bFinalResults = correct_output(a, n, value);


In [ ]:

  // release resources
  checkCudaErrors(cudaEventDestroy(start));
  checkCudaErrors(cudaEventDestroy(stop));
  checkCudaErrors(cudaFreeHost(a));
  checkCudaErrors(cudaFree(d_a));


In [ ]:

  exit(bFinalResults ? EXIT_SUCCESS : EXIT_FAILURE);